# Sentiment analysis COVID-19 scholar's articles

# Imports

In [15]:
import os
import json
import pandas as pd
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from  nltk import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.collocations import *
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm
import warnings
warnings.filterwarnings('ignore')
style.use('ggplot')
np.random.seed(0)

# Keras imports

In [16]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

In [17]:
ls

 Volume in drive C is OS
 Volume Serial Number is D816-28B0

 Directory of C:\Users\luigi\FlatironSchool\personal_project

17/04/2020  12:40    <DIR>          .
17/04/2020  12:40    <DIR>          ..
23/03/2020  14:33    <DIR>          .ipynb_checkpoints
23/03/2020  14:30    <DIR>          biorxiv_medrxiv
23/03/2020  14:13    <DIR>          comm_use_subset
21/03/2020  00:32            26,690 COVID.DATA.LIC.AGMT.pdf
17/04/2020  12:39            49,035 COVID-19.ipynb
23/03/2020  14:13    <DIR>          custom_license
21/03/2020  00:37             2,906 json_schema.txt
21/03/2020  00:37             1,574 metadata.readme
23/03/2020  14:14    <DIR>          noncomm_use_subset
17/04/2020  12:01        29,968,612 weights-improvement-01-9.0828.hdf5
17/04/2020  12:21        29,968,884 weights-improvement-20-5.2272.hdf5
               6 File(s)     60,017,701 bytes
               7 Dir(s)  12,534,362,112 bytes free


# See documents README

In [18]:
with open('metadata.readme', 'r') as f:
    data = f.read()
    print(data)

(1) Metadata for papers from these sources are combined: CZI, PMC, BioRxiv/MedRxiv. (total records 29500)
	- CZI 1236 records
	- PMC 27337
	- bioRxiv 566
	- medRxiv 361
(2) 17K of the paper records have PDFs and the hash of the PDFs are in 'sha'
(3) For PMC sourced papers, one paper's metadata can be associated with one or more PDFs/shas under that paper - a PDF/sha correponding to the main article, and possibly additional PDF/shas corresponding to supporting materials for the article.
(4)	13K of the PDFs were processed with fulltext ('has_full_text'=True)
(5) Various 'keys' are populated with the metadata:
	- 'pmcid': populated for all PMC paper records (27337 non null)
	- 'doi': populated for all BioRxiv/MedRxiv paper records and most of the other records (26357 non null)
	- 'WHO #Covidence': populated for all CZI records and none of the other records (1236 non null)
	- 'pubmed_id': populated for some of the records
	- 'Microsoft Academic Paper ID': populated for some of the records


# Text generation with LSTM Recurrent Neural Network

In [19]:
directory = ['comm_use_subset']

In [20]:
#documents= []
#for d in directory:
#        for index, file in enumerate(os.listdir(f"{d}/{d}")):
#            file_path = f"{d}/{d}/{file}"
#            j = json.load(open(file_path, 'rb'))
#            for x, y in j['bib_entries'].items():
#                title = y['title']
#                
#                documents.append(title.lower())

In [21]:
documents = []
for d in directory:
    for index, file in enumerate(os.listdir(f"{d}/{d}")):
        file_path = f"{d}/{d}/{file}"
        j = json.load(open(file_path, 'rb'))

        title = j['metadata']['title']

        documents.append(title.lower())

### Create a set of all of the distinct characters in the book, then create a map of each character to a unique integer

In [22]:
# create mapping of unique chars to integers
chars = sorted(list(set(documents)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [23]:
n_chars = len(documents)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  9118
Total Vocab:  8681


In [24]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = documents[i:i + seq_length]
    seq_out = documents[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  9018


In [25]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [46]:
print(dataX[1])

[8352, 6067, 8278, 4713, 3113, 3408, 7750, 1527, 5023, 7345, 6213, 7241, 455, 386, 2900, 2776, 1128, 4547, 8061, 2543, 2886, 6531, 6262, 7936, 5948, 3798, 4459, 256, 8662, 324, 7646, 1709, 532, 3263, 6440, 3694, 7073, 3188, 7647, 5878, 1151, 3366, 6011, 2872, 5504, 4397, 7974, 1052, 5534, 2865, 0, 334, 5015, 2511, 7881, 344, 113, 8388, 913, 1697, 5624, 7779, 5808, 7622, 4172, 6451, 3935, 1670, 4498, 4420, 3648, 2521, 3093, 0, 6904, 3422, 6672, 5658, 1343, 5363, 6936, 3451, 4351, 8539, 1234, 1025, 75, 7822, 6168, 6028, 6758, 8151, 2055, 7486, 6200, 1426, 1971, 7648, 212, 5649]


In [55]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [56]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min'))
callbacks_list = [checkpoint]

In [29]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
9018/9018 [==============================] - 23s 3ms/step - loss: 9.0705
Epoch 2/20
9018/9018 [==============================] - 24s 3ms/step - loss: 9.0611
Epoch 3/20
9018/9018 [==============================] - 26s 3ms/step - loss: 9.0526
Epoch 4/20
9018/9018 [==============================] - 27s 3ms/step - loss: 9.0214
Epoch 5/20
9018/9018 [==============================] - 27s 3ms/step - loss: 8.9593
Epoch 6/20
9018/9018 [==============================] - 28s 3ms/step - loss: 8.8496
Epoch 7/20
9018/9018 [==============================] - 27s 3ms/step - loss: 8.7551
Epoch 8/20
9018/9018 [==============================] - 26s 3ms/step - loss: 8.5076
Epoch 9/20
9018/9018 [==============================] - 26s 3ms/step - loss: 8.1644
Epoch 10/20
9018/9018 [==============================] - 26s 3ms/step - loss: 7.7269
Epoch 11/20
9018/9018 [==============================] - 26s 3ms/step - loss: 7.1374
Epoch 12/20
9018/9018 [==============================] - 27s 3ms/step - lo

In [30]:
# load the network weights
filename = "weights-improvement-20-5.2272.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [31]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [47]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("-----------------------------")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" A permanent host shift of rabies virus from Chiroptera to Carnivora associated with recombination OPENMolecular Sciences Therapeutic Applications of Rose Hips from Different Rosa SpeciesBMC Bioinformatics An effective docking strategy for virtual screening based on multi-objective optimization algorithmA mechanistic spatio-temporal framework for modelling individual-to-individual transmission-With an application to the 2014-2015 West Africa Ebola outbreakRespiratory Viral Infection-Induced Microbiome Alterations and Secondary Bacterial PneumoniaFeline Infectious Peritonitis Virus Nsp5 Inhibits Type I Interferon Production by Cleaving NEMO at Multiple SitesTargeted Collection of Plasmid DNA in Large and Growing Animal Muscles 6 Weeks after DNA Vaccination with and without ElectroporationSupporting Information: Demographic and environmental drivers of metagenomic viral diversity in vampire batsThe Role of Chemokines during Viral Infection of the CNSMethanolic Extract of Rhizoma C

Detection and characterization of respiratory viruses causing acute respiratory illness and asthma exacerbation in children during three different seasons (2011-2014) in Mexico CityARTICLE Protein nanovaccine confers robust immunity against ToxoplasmaARTICLE Protein nanovaccine confers robust immunity against ToxoplasmaARTICLE Protein nanovaccine confers robust immunity against ToxoplasmaRhinovirus Genome Variation during Chronic Upper and Lower Respiratory Tract InfectionsDevelopment of nsP2 protease based cell free high throughput screening assay for evaluation of inhibitors against emerging Chikungunya virus OPENDevelopment of nsP2 protease based cell free high throughput screening assay for evaluation of inhibitors against emerging Chikungunya virus OPENDevelopment of nsP2 protease based cell free high throughput screening assay for evaluation of inhibitors against emerging Chikungunya virus OPENInvolvement of P2X7 receptor signaling on regulating the differentiation of Th17 cells 

Antiviral Lead Compounds from Marine SpongesAntiviral Lead Compounds from Marine SpongesArticle 120-W (2019) Porcine Torovirus (PToV)-A Brief Review of EtiologyPhylodynamic applications in 21 st century global infectious disease researchRhinovirus Genome Variation during Chronic Upper and Lower Respiratory Tract InfectionsRhinovirus Genome Variation during Chronic Upper and Lower Respiratory Tract InfectionsMolecular Sciences Two Sides of the Coin: Ezrin/Radixin/Moesin and Merlin Control Membrane Structure and Contact InhibitionTranscriptional and Translational Landscape of Equine Torovirus GENOME REPLICATION AND REGULATION OF VIRAL GENE EXPRESSION crossmGenetic diversity of BCoV in Brazilian cattle herdsGenetic diversity of BCoV in Brazilian cattle herdsInhibition of enterovirus 71 replication and viral 3C protease by quercetinAntiviral Lead Compounds from Marine SpongesAntiviral Lead Compounds from Marine SpongesArticle 120-W (2019) Porcine Torovirus (PToV)-A Brief Review of Etiology

Genetic diversity of BCoV in Brazilian cattle herdsGenetic diversity of BCoV in Brazilian cattle herdsInhibition of enterovirus 71 replication and viral 3C protease by quercetinAntiviral Lead Compounds from Marine SpongesAntiviral Lead Compounds from Marine SpongesArticle 120-W (2019) Porcine Torovirus (PToV)-A Brief Review of EtiologyPhylodynamic applications in 21 st century global infectious disease researchRhinovirus Genome Variation during Chronic Upper and Lower Respiratory Tract InfectionsRhinovirus Genome Variation during Chronic Upper and Lower Respiratory Tract InfectionsMolecular Sciences Two Sides of the Coin: Ezrin/Radixin/Moesin and Merlin Control Membrane Structure and Contact InhibitionTranscriptional and Translational Landscape of Equine Torovirus GENOME REPLICATION AND REGULATION OF VIRAL GENE EXPRESSION crossmGenetic diversity of BCoV in Brazilian cattle herdsGenetic diversity of BCoV in Brazilian cattle herdsInhibition of enterovirus 71 replication and viral 3C prot

Phylodynamic applications in 21 st century global infectious disease researchRhinovirus Genome Variation during Chronic Upper and Lower Respiratory Tract InfectionsRhinovirus Genome Variation during Chronic Upper and Lower Respiratory Tract InfectionsMolecular Sciences Two Sides of the Coin: Ezrin/Radixin/Moesin and Merlin Control Membrane Structure and Contact InhibitionTranscriptional and Translational Landscape of Equine Torovirus GENOME REPLICATION AND REGULATION OF VIRAL GENE EXPRESSION crossmGenetic diversity of BCoV in Brazilian cattle herdsGenetic diversity of BCoV in Brazilian cattle herdsInhibition of enterovirus 71 replication and viral 3C protease by quercetinAntiviral Lead Compounds from Marine SpongesAntiviral Lead Compounds from Marine SpongesArticle 120-W (2019) Porcine Torovirus (PToV)-A Brief Review of EtiologyPhylodynamic applications in 21 st century global infectious disease researchRhinovirus Genome Variation during Chronic Upper and Lower Respiratory Tract Infect

In [36]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [37]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
9018/9018 [==============================] - 74s 8ms/step - loss: 9.0702
Epoch 2/20
9018/9018 [==============================] - 80s 9ms/step - loss: 9.0891
Epoch 3/20
9018/9018 [==============================] - 81s 9ms/step - loss: 9.0637
Epoch 4/20
9018/9018 [==============================] - 81s 9ms/step - loss: 8.9977
Epoch 5/20
9018/9018 [==============================] - 81s 9ms/step - loss: 8.9651
Epoch 6/20
9018/9018 [==============================] - 80s 9ms/step - loss: 8.9502
Epoch 7/20
9018/9018 [==============================] - 80s 9ms/step - loss: 8.9377
Epoch 8/20
9018/9018 [==============================] - 81s 9ms/step - loss: 8.9243
Epoch 9/20
9018/9018 [==============================] - 80s 9ms/step - loss: 8.9063
Epoch 10/20
9018/9018 [==============================] - 80s 9ms/step - loss: 8.9067
Epoch 11/20
9018/9018 [==============================] - 81s 9ms/step - loss: 8.8754
Epoch 12/20
9018/9018 [==============================] - 79s 9ms/step - lo

In [60]:
# load the network weights
filename = "models/weights-improvement-20-6.1178.hdf5.model"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

AttributeError: 'NoneType' object has no attribute 'load_weights'

In [61]:
model.summary()

AttributeError: 'NoneType' object has no attribute 'summary'

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
print("-----------------------------")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

In [19]:
#df_bibl_titles.drop(df_bibl_titles.head(14).index,inplace=True) # drop last n rows
#df_bibl_titles.head()

,title
14,The dendritic cell system and its role in immu...
15,Immunobiology of dendritic cells
16,The CD4-centered universe of human T cell subsets
17,Ligation of CD40 on dendritic cells triggers p...
18,High level IL-12 production by murine dendriti...


# Sentiment analysis

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
compound_list = []
for x in df_for_series['title']:
    ss = sid.polarity_scores(x)
    compound_list.append(ss['compound'])

In [ ]:
df_for_series['compound'] = compound_list
df_for_series.head()

In [ ]:
df_for_series.tail()

In [ ]:
len(df_for_series)

In [ ]:
#df_for_series = df_for_series[df_for_series.index <= '2020.0']

In [ ]:
df_for_series['year'].describe()

In [ ]:
df_for_series.set_index('year', inplace=True)
df_for_series.head()

In [ ]:
df_for_series = df_for_series.sort_index()
df_for_series.head()

In [ ]:
df_for_series['compound'].describe()

In [ ]:
target_list = []
for x in df_for_series['compound']:
    if x < -.25:
        target_list.append(-1)
    elif x > .25:
        target_list.append(1)
    else: 
        target_list.append(0)

In [ ]:
df_for_series['target'] = target_list
df_for_series.head(20)

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(compound_list);

In [ ]:
df_time_series = df_for_series[['compound']].copy()
df_time_series.tail()

Drop outliers

In [ ]:
#df_time_series = df_time_series.loc[df_time_series.index < '2020.0']

In [ ]:
df_time_series.plot(kind='line', figsize=(12,8))
plt.title('Compound/Time', fontsize=28)
plt.xlabel('Time', fontsize=24)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel('Compound', fontsize=24)
plt.legend(fontsize=20);

In [ ]:
# rolling mean

# ARIMA

In [ ]:
df_time_series['date'] = pd.to_datetime(df_time_series.index, unit='M')
df_time_series.tail()

In [ ]:
def arimamodel(timeseries):
    '''
     Creating the model: automatic ARIMA using ADF to test stationarity, 
     start values for p and q are set to 1, and the crypto data is not assumed to be seasonal 
 
     Args: 
         timeseries: Pandas timeseries with closing price 
         
     Returns: 
         automodel: automatic ARIMA model
    '''
    automodel = pm.auto_arima(df, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              seasonal=False,
                              trace=True)
    return automodel

In [ ]:
def plotarima(n_periods, timeseries, automodel):
    ''' 
    Plot ARIMA forecast using statsmodel’s plot_predict, 
        filling the area between the upper and lower prediction bounds.
    
    Args: 
        n_periods: steps for the forecast
        timeseries: Pandas timeseries with closing price 
        automodel: automatic ARIMA model
    '''
    # Hourly index
    fc_ind = pd.date_range(df.index[df.shape[0]-1], 
                           periods=n_periods, freq="h")
    # Forecast series
    fc_series = pd.Series(forecast[0], index=fc_ind)
    # Upper and lower confidence bounds
    lower_series = pd.Series(forecast[2][:,0], index=fc_ind)
    upper_series = pd.Series(forecast[2][:,1], index=fc_ind)
    # Create plot
    plt.figure(figsize=(20,10))
    plt.plot(df)
    plt.plot(fc_series, color="blue")
    plt.xlabel("date")
    plt.ylabel(df.name)
    plt.fill_between(lower_series.index, 
                     lower_series, 
                     upper_series, 
                     color="k", 
                     alpha=0.25)
    plt.legend(("past", "forecast", "95% confidence interval"),  
               loc="upper left")
    plt.show();

In [ ]:
automodel = arimamodel(df_time_series)

In [ ]:
# Build Model
model = ARIMA(train, order=(0, 1, 0))  
model_fit = model.fit()  

# Forecast
fc, se, conf = model_fit.forecast(steps=72, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=test.index)
lower_series = pd.Series(conf[:, 0], index=test.index)
upper_series = pd.Series(conf[:, 1], index=test.index)

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.xticks(rotation=45)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Forecast vs Actuals')
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
automodel.summary()

In [ ]:
automodel.plot_diagnostics(figsize=(15, 18))
plt.show()

In [ ]:
print(model_fit.summary())

# Create df for commercial articles

In [ ]:
def create_df_text(directory): 
    
    ''' Create df with title, abstract and full text as columns
    
        Argument: directory: choose the directory needed
        
        Return: dataframe
    
    '''
    docs = []
    for d in directory:
        for index, file in enumerate(os.listdir(f"{d}/{d}")):
            file_path = f"{d}/{d}/{file}"
            j = json.load(open(file_path, 'rb'))

            title = j['metadata']['title']

            try:
                abstract = j['abstract'][0]['text']
            except:
                abstract = ""

            full_text = ""   
            for text in j['body_text']:
                full_text += text['text'] + "\n\n"


            docs.append([title, abstract, full_text])

            if index % 10 == 0:
                print(index, end=" ")
                
    df = pd.DataFrame(docs, columns = ['title', 'abstract', 'full_text'])
            
    return df

In [ ]:
df_comm = create_df_text(directory)

In [ ]:
df_comm.head()

In [ ]:
df_comm.isnull().any()

# Check articles that contain the word incubation

In [ ]:
incubation = df_comm[df_comm['full_text'].str.contains('incubation')]
incubation.head()

Let's write a regex for capture digits followed by the word day and save in a list.

In [ ]:
incubation_time = []
texts = incubation['full_text'].values

for text in texts:
    for sentence in text.split(". "):
        if "incubation" in sentence:
            single_day = re.findall(r'( \d{1,2}(\.\d{1,2})? day[s]?)', sentence)
            
            if len(single_day) == 1:
                num = single_day[0][0].split(" ")
                incubation_time.append(float(num[1]))

In [ ]:
print(len(incubation_time))
print(incubation_time, end=' ')

I'm assuming that values over 30 are outliers so I'm gonna remove them.

In [ ]:
inc_time = [i for i in incubation_time if i < 30]

In [ ]:
df_inc = pd.DataFrame(inc_time, columns=['Incubation days'])
df_inc.head()

In [ ]:
df_inc.describe()

### The mean of the commercial articles for the incubation days is 7 days.

In [ ]:
plt.figure(figsize=(20,10))
plt.hist(inc_time)
plt.title('Incubatione time', fontsize=24)
plt.ylabel('Bin counts', fontsize=20)
plt.xlabel('Days', fontsize=20);

# Create df for non commercial articles

In [ ]:
directory = ['noncomm_use_subset']

In [ ]:
df_non_comm = create_df_text(directory)

In [ ]:
df_non_comm.head()

In [ ]:
df_non_comm.isnull().any()

# Check articles that contain the word incubation

In [ ]:
incubation = df_non_comm[df_non_comm['full_text'].str.contains('incubation')]
incubation.head()

Let's write a regex for capture digits followed by the word day and save in a list.

In [ ]:
incubation_time = []
texts = incubation['full_text'].values

for text in texts:
    for sentence in text.split(". "):
        if "incubation" in sentence:
            single_day = re.findall(r'( \d{1,2}(\.\d{1,2})? day[s]?)', sentence)
            
            if len(single_day) == 1:
                num = single_day[0][0].split(" ")
                incubation_time.append(float(num[1]))

In [ ]:
print(len(incubation_time))
print(incubation_time, end=' ')

I'm assuming that values over 30 are outliers so I'm gonna remove them.

In [ ]:
inc_time = [i for i in incubation_time if i < 30]

In [ ]:
df_inc = pd.DataFrame(inc_time, columns=['Incubation days'])
df_inc.head()

In [ ]:
df_inc.describe()

### The mean of the non commercial articles for the incubation days is 9 days.

In [ ]:
plt.figure(figsize=(20,10))
plt.hist(inc_time)
plt.title('Incubatione time', fontsize=24)
plt.ylabel('Bin counts', fontsize=20)
plt.xlabel('Days', fontsize=20);

# Preprocessing the Data

# Stopword and punctualization

In [ ]:
# Get all the stop words in the English language
stopwords_list = stopwords.words('english')
stopwords_list = stopwords_list + ['the', 'a', 'in', 'al', 'c', 'et','e', 'study', 'virus', 'b','p', 
                      'using', 'also', 'infection', 'cell', 'h', 'n', 'protein']

In [ ]:
# Now we have a list that includes all english stopwords, as well as all punctuation
stopwords_list += list(string.punctuation)

# Tokenizing our Data

In [ ]:
def tokenize(text):
    joined_text = ' '.join(text)
    tokenized_text = word_tokenize(joined_text)
    pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
    text_tokens_raw = nltk.regexp_tokenize(str(tokenized_text), pattern)
    stopped_tokens = [w.lower() for w in text_tokens_raw if w not in stopwords_list]
    
    return stopped_tokens

In [ ]:
full_txt_list = []
for x in df_non_comm['full_text']:
    full_txt_list.append(x)

In [ ]:
tokenized_full_txt = tokenize(full_txt_list)

In [ ]:
tokenized_full_txt[:10]

# Count Vectorization.

In [ ]:
def count_vectorize(text, vocab=None):
    if vocab:
        unique_words = vocab
    else:
        unique_words = list(set(text))
    
    txt_dict = {i:0 for i in unique_words}
    
    for word in text:
        txt_dict[word] += 1
    
    return txt_dict

In [ ]:
test_vectorized = count_vectorize(tokenized_full_txt)
print(test_vectorized)

# Get the 50 most common words 

In [ ]:
freqdist = FreqDist(tokenized_full_txt)

# get the 50 most common words 
most_common = freqdist.most_common(50)
print(most_common[:20], end = ' ')

In [ ]:
num_to_show = 30

labels = [" ".join(e[0]) for e in freqdist.most_common(num_to_show)]
values = [e[1] for e in freqdist.most_common(num_to_show)]

indexes = np.arange(len(labels))
width = 1
plt.figure(figsize=(20,10))
plt.barh(indexes, values, width)
plt.yticks(indexes + width * 0.2, labels, fontsize=20)
plt.show()

# Normalized Word Frequency

In [ ]:
total_word_count = sum(freqdist.values())
titles_top_200 = freqdist.most_common(200)
print('Word\t\t\tNormalized Frequency')
for word in titles_top_200:
    normalized_frequency = word[1] / total_word_count
    print('{} \t\t\t {:.4}'.format(word[0], normalized_frequency))

# Bigrams

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
title_finder = BigramCollocationFinder.from_words(tokenized_full_txt)
title_scored = title_finder.score_ngrams(bigram_measures.raw_freq)
title_scored[:50]

# Pointwise Mutual Information

In [ ]:
title_finder.apply_freq_filter(5)
title_pmi_scored = title_finder.score_ngrams(bigram_measures.pmi)
title_pmi_scored

# TF-IDF Vectorization

If the word is used often in many other documents, it is not unique, and therefore probably not too useful if we wanted to figure out how this document is unique in relation to other documents. Conversely, if a word is used many times in a document, but rarely in all the other documents we are considering, then it is likely a good indicator for telling us that this word is important to the document in question.

# TF

In [ ]:
def term_frequency(BoW_dict):
    total_word_count = sum(BoW_dict.values())
    
    for ind, val in BoW_dict.items():
        BoW_dict[ind] = val/ total_word_count
    
    return BoW_dict

In [ ]:
test = term_frequency(test_vectorized)
print(list(test)[:20])

# IDF

In [ ]:
def inverse_document_frequency(list_of_dicts):
    vocab_set = set()
    
    # Iterate through list of dfs and add index to vocab_set
    for d in list_of_dicts:
        for word in d.keys():
            vocab_set.add(word)
    
    # Once vocab set is complete, create an empty dictionary with a key for each word and value of 0.
    full_vocab_dict = {i:0 for i in vocab_set}
    
    # Loop through each word in full_vocab_dict
    for word, val in full_vocab_dict.items():
        docs = 0
        
        # Loop through list of dicts.  Each time a dictionary contains the word, increment docs by 1
        for d in list_of_dicts:
            if word in d:
                docs += 1
        
        # Now that we know denominator for equation, compute and set IDF value for word
        
        full_vocab_dict[word] = np.log((len(list_of_dicts)/ float(docs)))
    
    return full_vocab_dict

# Computing TF-IDF

In [ ]:
def tf_idf(list_of_dicts):
    # Create empty dictionary containing full vocabulary of entire corpus
    doc_tf_idf = {}
    idf = inverse_document_frequency(list_of_dicts)
    full_vocab_list = {i:0 for i in list(idf.keys())}
    
    # Create tf-idf list of dictionaries, containing a dictionary that will be updated for each document
    tf_idf_list_of_dicts = []
    
    # Now, compute tf and then use this to compute and set tf-idf values for each document
    for doc in list_of_dicts:
        doc_tf = term_frequency(doc)
        for word in doc_tf:
            doc_tf_idf[word] = doc_tf[word] * idf[word]
        tf_idf_list_of_dicts.append(doc_tf_idf)
    
    return tf_idf_list_of_dicts

# Vectorizing All Documents

In [ ]:
count_vectorized_all_documents = []
count_vectorized_all_documents.append(test_vectorized)

In [ ]:
tf_idf_all_docs = tf_idf(count_vectorized_all_documents)
print(list(tf_idf_all_docs[0])[:20])

In [ ]:
num_dims = len(tf_idf_all_docs[0])
print("Number of Dimensions: {}".format(num_dims))

# Sentiment analysis title sentences

In [ ]:
title_list = []
for x in df_non_comm['title']:
    title_list.append(x)

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
compound_list = []
for x in title_list:
    ss = sid.polarity_scores(x)
    compound_list.append(ss['compound'])

In [ ]:
compound_list

In [ ]:
df_title_compound = pd.DataFrame(title_list,columns=['title'])
df_title_compound.head()

In [ ]:
df_title_compound['compound'] = compound_list
df_title_compound.head()

In [ ]:
df_title_compound['compound'].describe()

In [ ]:
df_title_compound.isnull().any()

In [ ]:
target_list = []
for x in df_title_compound['compound']:
    if x < -.25:
        target_list.append(-1)
    elif x > .25:
        target_list.append(1)
    else: 
        target_list.append(0)

In [ ]:
df_title_compound['target'] = target_list
df_title_compound.head(20)

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(compound_list);

# word_cloud

In [ ]:
neg_title = df_title_compound[df_title_compound['target'] == -1]
neg_title.head()

In [ ]:
neg_string = []
for t in neg_title['title']:
    neg_string.append(t)
neg_string

# Negative sentiment

In [ ]:
neg_title = df_title_compound[df_title_compound['target'] == -1]
neg_string = []
for t in neg_title['title']:
    neg_string.append(t)
neg_string = pd.Series(neg_string).str.cat(sep=' ')

wordcloud = WordCloud(width=1600, height=800,max_font_size=200,colormap='magma').generate(neg_string)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Positive sentiment

In [ ]:
pos_title = df_title_compound[df_title_compound['target'] == 1]
pos_string = []
for t in pos_title['title']:
    pos_string.append(t)
pos_string = pd.Series(pos_string).str.cat(sep=' ')

wordcloud = WordCloud(width=1600, height=800,max_font_size=200).generate(pos_string)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Neutral sentiment

In [ ]:
neut_title = df_title_compound[df_title_compound['target'] == 0]
neut_string = []
for t in neut_title['title']:
    neut_string.append(t)
neut_string = pd.Series(neut_string).str.cat(sep=' ')

wordcloud = WordCloud(width=1600, height=800,max_font_size=200).generate(neut_string)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Sentiment analysis words

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
list_x = []
for x in most_common:
    print(x[0])

In [ ]:
comp_word_list = []
for x in most_common:
    ss = sid.polarity_scores(x[0])
    comp_word_list.append(ss['compound'])

In [ ]:
comp_word_list

# What is known about transmission?

In [ ]:
df = pd.DataFrame(full_txt_list, columns=['full_text'])
df.head(20)

In [ ]:
transmission = df[df['full_text'].str.contains('transmission')]
len(transmission)

In [ ]:
transm_text = transmission.full_text.values

In [ ]:
transm_list = []
for t in transm_text:
    if 'transmission' in t:
        transm_list.append(t)

In [ ]:
for t in transm_text:
    if 'transmission' in t:
        transm_list.append(t)

# NLP-architect